In [1]:
import os
from glob import glob
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from dotenv import load_dotenv
load_dotenv()

import textwrap
from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Initialize variables
documents = []
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


#print(OPENAI_API_KEY)
# Define the directory containing the PDF files
pdf_directory = './data'

자료를 벡터 db에 넣기

In [2]:


# pdf를 사용해서 pdf(논문)을 모두 로드
pdf_files = glob(os.path.join(pdf_directory, '*.pdf'))

# Load all PDF files using PyPDFLoader
for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pdf_documents = loader.load()
    documents.extend(pdf_documents)
    
# 텍스트는 RecursiveCharacterTextSplitter를 사용하여 분할
chunk_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = chunk_splitter.split_documents(documents)
print(documents)

# embeddings은 OpenAI의 임베딩을 사용
# vectordb는 chromadb사용함

embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings)
retriever = vectordb.as_retriever()

[Document(page_content='학부(과): ICT공학부 전공:소프트웨어전공 성명: 박은빈 학번:202004052개인별 수강 과목 리스트\n이수구분 학수번호 교과명 학점점수성적년도학년학기학점중복신청 구분 개설전공\n자선 NG21402 자기관리와진로탐색 1P2020 12 교양\n자선 학점 합계 : 1.0 학점\n타전VG34202 강화학습 399A+2023 32 인공지능전공\nVG24201 데이터시각화 399A+2023 32 데이터사이언스전공\nVG34201 딥러닝기초 397A+2023 32 인공지능전공\n타전 학점 합계 : 9.0 학점\n기초ND01609 Academic English I(intro) 395A+2020 11 소프트웨어응용학부\nND01607 Academic English II(intro) 392A2020 12 소프트웨어응용학부\nNA01109 글쓰기 298A+2020 12 소프트웨어응용학부\nND01601 기독교와현대사회 299A+2020 12 소프트웨어응용학부\nVA01601 인성과학문I 0P2020 11 소프트웨어응용학부\nVA01601 인성과학문II 0P2020 12 소프트웨어응용학부\nVA01601 인성과학문III 0P2021 21 소프트웨어응용학부\nVA01601 인성과학문IV 0P2021 22 소프트웨어응용학부\nND01612 채플(공동체리더십)III 0P2021 21 소프트웨어응용학부\nND01604 채플(이웃사랑)II 0P2020 12 소프트웨어응용학부\nND01605 채플(인성교육)I 0P2020 11 음악학과\nND01612 채플(행복나눔)IV 0P2021 22 소프트웨어응용학부\nND01602 컴퓨터프로그래밍 395A+2020 11 소프트웨어응용학부\n기초 학점 합계 : 13.0 학점\n균형NE31705 디지털과빅데이터 398A+2023 31 교양(3영역)\nNE11705 실존적삶과철학 3100A+2023 31 교양(1영역)\nNE41707 알기쉬운자연과학 398A+2023 32 교양(4영역)\nNE61702 인간과

### 프롬프트

In [3]:
SYS_PROMPT = f"""
    너는 사용자가 강남대학교에서 졸업할 수 있는지 등을 물어보았을때 성실하게 답변해주는 학사지원 인공지능 챗봇이야.
    사용자의 수강정보는 강남대학교의 졸업요건과 함께 데이터베이스에 같이 저장되어 있어.
    주전공, 복수전공, 등을 나눠서 설명해줘야해
    
"""

INPUT_PROMPT = f"""
라고 대답했습니다. \\
이 사용자가 가지고 있는 한국형 외로움을 설명해주세요. \\
그리고 어떤 대화 상대가 되어주어야 하는지 출력해주세요
"""

모델 선언

In [4]:
# 필요한 라이브러리 및 모듈을 임포트합니다.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 프롬프트 템플릿을 정의합니다.
# SYS_PROMPT는 시스템 메시지로, 템플릿에 포함됩니다. 
# {context}와 {question}은 실행 시 동적으로 채워질 자리표시자입니다.
template = SYS_PROMPT + '''Answer the question based only on the following cont ext:
{context}

Question: {question}
'''

# ChatPromptTemplate.from_template() 메서드를 사용하여 프롬프트 템플릿을 생성합니다.
prompt = ChatPromptTemplate.from_template(template)

# ChatOpenAI 인스턴스를 생성하여 LLM (대규모 언어 모델)을 설정합니다.
# 여기서는 'gpt-4o' 모델을 사용하고, temperature는 0으로 설정하여 출력의 일관성을 높입니다.
model = ChatOpenAI(model='gpt-4o', temperature=0)

# 문서들을 형식화하는 함수를 정의합니다.
# 각 문서의 페이지 내용을 합쳐 하나의 문자열로 반환합니다.
def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)

# RAG (Retrieval-Augmented Generation) 체인을 연결합니다.
# 이 체인은 문서 검색, 형식화, 프롬프트 적용, 모델 호출, 출력 파싱의 과정을 거칩니다.
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}  # 'context'는 retriever와 format_docs를 통해 설정되고, 'question'은 그대로 전달됩니다.
    | prompt  # 프롬프트 템플릿을 적용합니다.
    | model  # 모델을 호출합니다.
    | StrOutputParser()  # 출력 파서를 통해 모델의 출력을 문자열로 변환합니다.
)

# 체인을 실행합니다.
# 입력 메시지는 질문과 답변 형식의 텍스트입니다.
input_message =  """
나 졸업 언제쯤 가능할꺼같아?
"""   # 추가적인 입력 프롬프트가 이어집니다.

# to_markdown() 함수를 호출하여 체인의 결과를 마크다운 형식으로 변환합니다.
to_markdown(rag_chain.invoke("input_message"))


> 박은빈 학생의 졸업 가능 여부를 확인하기 위해서는 다음과 같은 졸업 요건을 충족해야 합니다:
> 
> 1. **학점 요건**:
>    - 졸업 기준 학점은 130학점입니다.
>    - 현재 박은빈 학생은 117학점을 취득한 상태입니다. 따라서 추가로 13학점을 더 이수해야 합니다.
> 
> 2. **전공 요건**:
>    - 박은빈 학생은 소프트웨어 전공의 심화전공을 이수 중입니다.
>    - 전공 필수 과목 및 전공 선택 과목을 모두 이수해야 합니다.
>    - 현재 전공 과목에서 57학점을 이수한 상태입니다.
> 
> 3. **교양 및 기타 요건**:
>    - 균형교양, 기초교양 등 교양 과목의 이수 요건을 충족해야 합니다.
>    - 사회봉사 인증을 완료해야 합니다. 박은빈 학생은 사회봉사 인증을 완료한 상태입니다.
> 
> 4. **평점 요건**:
>    - 전체 평점 평균은 4.38로 매우 우수한 상태입니다.
> 
> 박은빈 학생은 현재 117학점을 이수하였으며, 졸업을 위해서는 추가로 13학점을 더 이수해야 합니다. 또한, 전공 필수 및 선택 과목, 교양 과목의 이수 요건을 모두 충족해야 합니다. 현재까지의 성적과 이수 현황을 보면, 추가 학점을 이수하고 나면 졸업 요건을 충족할 가능성이 높습니다.